## Special Functions - col and lit

Let us understand special functions such as col and lit.

* First let us create Data Frame for demo purposes.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [ ]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Processing Column Data'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

In [ ]:
employees = [(1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                ]

In [ ]:
employeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, nationality STRING,
                    phone_number STRING, ssn STRING"""
                   )

* For Data Frame APIs such as `select`, `groupBy`, `orderBy` etc we can pass column names as strings.

In [ ]:
from pyspark.sql.functions import col

In [ ]:
employeesDF. \
    select(col("first_name"), col("last_name")). \
    show()

In [ ]:
# Alternative by passing column names as strings.
employeesDF. \
    select("first_name", "last_name"). \
    show()

* If there are no transformations on any column in any function then we should be able to pass all column names as strings.
* If not we need to pass all columns as type column by using col function or its shorthand operator $.

In [ ]:
employeesDF. \
    groupBy("nationality"). \
    count(). \
    show()

In [ ]:
employeesDF. \
    orderBy("employee_id"). \
    show()

* However, if we want to apply any transformation using functions then passing column names as strings to some of the functions will not suffice. We have to pass them as column type.

In [ ]:
from pyspark.sql.functions import upper

In [ ]:
employeesDF. \
    select(upper("first_name"), upper("last_name")). \
    show()

* `col` is the function which will convert column name from string type to **Column** type. We can also refer column names as **Column** type using Data Frame name.

In [ ]:
from pyspark.sql.functions import col, upper

In [ ]:
employeesDF. \
    select(upper(col("first_name")), upper(col("last_name"))). \
    show()

In [ ]:
employeesDF. \
    groupBy(upper(col("nationality"))). \
    count(). \
    show()

In [ ]:
employeesDF. \
    orderBy(col("first_name").desc()). \
    show()

In [ ]:
# Alternative - we can also refer column names using Data Frame like this
employeesDF. \
    orderBy(upper(employeesDF['first_name']).alias('first_name')). \
    show()

In [ ]:
# Alternative - we can also refer column names using Data Frame like this
employeesDF. \
    orderBy(upper(employeesDF.first_name).alias('first_name')). \
    show()

* Sometimes, we want to add a literal to the column values. For example, we might want to concatenate first_name and last_name with separated by comma and space in between.

In [ ]:
from pyspark.sql.functions import concat

In [ ]:
# Same as above
employeesDF. \
    select(concat(col("first_name"), ", ", col("last_name"))). \
    show()

In [ ]:
# Referring columns using Data Frame
employeesDF. \
    select(concat(employeesDF("first_name"), ", ", employeesDF("last_name"))). \
    show()

* If we pass the literals directly in the form of string or numeric type, then it will fail. We have to convert literals to column type by using `lit` function.

In [ ]:
from pyspark.sql.functions import concat, col, lit

employeesDF. \
    select(concat(col("first_name"), 
                  lit(", "), 
                  col("last_name")
                 ).alias("full_name")
          ). \
    show(truncate=False)